In [23]:
import requests as rq
from bs4 import BeautifulSoup as bs
from re import findall, sub, compile
from IPython.core.debugger import Tracer
import sys, os, csv
import PIL

In [43]:
#get list of people with wikipedia pages
wiki = "https://en.wikipedia.org/w/index.php?title=Category:Living_people"
wikibase = "https://en.wikipedia.org"

rp = rq.get(wiki)

people = []

while 1:
    
    #make sure we successfully got person
    if rp.status_code != 200:
        print("error retrieving page")
        break
        
    #make it a beautiful soup doc
    soup = bs(rp.text, 'lxml')
    
    #extract all people
    page_people = []
    p = soup.find_all(class_="mw-category")
    
    a_people = p[0].find_all("a")
    
    for a in a_people:
        page_people += [a.get_text()]
    
    people += page_people
    
    living = soup.find(id="mw-pages")
    next_ = living.find_all('a', text="next page")
    #print(next_[0]["href"])
    
    
    if not next_[0] is None:
        rp = rq.get(wikibase + next_[0]['href'])
    else:
        break

KeyboardInterrupt: 

In [61]:
#save list of figures
with open("names.txt",'w') as f:
    for name in people[:100000]:
        f.write('{0:s},\n'.format((name)))

In [10]:
#load csv of names
people = []
with open("names.txt","r") as f:
    csvlist = csv.reader(f)
    for row in csvlist:
        people+=[row[0]]

In [25]:
def get_wiki(name):
    

    headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:57.0) Gecko/20100101 Firefox/57.0"}
    
    #query string for a name
    base_query = "https://en.wikipedia.org"
    query = base_query + "/wiki/" + sub(' ','_',name)
    
    #get http and parse
    rp = rq.get(query,headers=headers)
    if rp.status_code != 200:
        print("Failed to query wikipedia for: %s" % name)
        return None
    soup = bs(rp.text,'lxml')
    
    #check to see if we have an image
    images = [name] #name will identify group of images
    infobox = soup.find(class_=["infobox vcard","infobox biography vcard"])
    if infobox is None:
        return None
    img = infobox.find('img')
    if img and img.has_attr("src"):
        images+=[img.get("src")]
    else:
        return None
    
    #check if parents are listed and there are 2
    parents = soup.find(text="Parents")
    if not parents:
        return None
    parents = parents.parent.parent
    parent_tags = parents.find_all('a')
                    
    if len(parent_tags) != 2:
        return None
    
    parent_urls = []
    
    for link in parent_tags:
        if not link.has_attr("href"):
            return None
        parent_urls += [link.get("href")]

    #we know we have two parents with wiki pages, now we want to check for images
    for url in parent_urls:
        #Tracer()()
        parent_rp = rq.get(base_query + url, headers=headers)
        if rp.status_code != 200:
            print("Failed to query wikipedia for: %s" % name)
            return None
        p_soup = bs(parent_rp.text,'lxml')
        infobox = p_soup.find(class_=["infobox vcard", "infobox biography vcard"])
        if infobox is None:
            return None
        img = infobox.find('img')
        if img and img.has_attr("src"):
            images+=[img.get("src")]
        else:
            return None
        
    
    #now we have urls for 3 urls: child, father, mother
    return images
    
    
    
#get spouse picture. returns none if no picture
def get_spouse(name, is_child):
    
    if is_child:
        query = "https://www.google.com/search?q=" + re.sub(' ', '+', name) + "+child"
    else:
        query = "https://www.google.com/search?q=" + re.sub(' ', '+', name) + "+spouse"
    
    rp = rq.get(query)
    
    if rp.status_code != 200:
        print("Failed to query google for: %s" % name)
        return None
    
    soup = bs(rp.text, 'lxml')
    
    #did we get a panel with info above search results?
    if soup.findall(class_='kp-header') or soup.findall(class_=re.compile("bfj_*")):
        return soup.find(id="uid_dimg_0")
    
    #otherwise we couldnt find a spouse
    else:
        return None
    
def get_person(name):
    if is_child:
        query = "https://www.google.com/search?q=" + re.sub(' ', '+', name) + "+child"
    else:
        query = "https://www.google.com/search?q=" + re.sub(' ', '+', name) + "+spouse"


In [ ]:
ex=["Donald Trump", "Barack Obama", "George W. Bush", "Jon Snow", "Philamander"]
photo_urls = [get_wiki(name) for name in people[:10000]]

Failed to query wikipedia for: Mohammad Aamer (cricketer
Failed to query wikipedia for: Mohammad Aamer (cricketer
Failed to query wikipedia for: Mohammad Aamer (cricketer


In [28]:
#count number of successful retrievals of urls
ex_photo_urls = []
for i,url in enumerate(photo_urls):
    if not url is None:
        ex_photo_urls += [[i] + url]
        

def prof_dl(families):
    for family in families:
        os.system('wget ' + family[2][2:] + ' ./' + str(family[0]) + '/child.jpg')
        os.system('wget ' + family[3][2:] + ' ./' + str(family[0]) + '/father.jpg')
        os.system('wget ' + family[4][2:] + ' ./' + str(family[0]) + '/mother.jpg')
        
len(ex_photo_urls)
    

0

In [29]:
photo_urls

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [85]:
#take list of urls and write 

False

In [19]:
parent_rp = rq.get("https://en.wikipedia.org/wiki/Mary_Anne_MacLeod_Trump")
if parent_rp.status_code != 200:
    print("Failed to query wikipedia for: %s" % name)
p_soup = bs(parent_rp.text,'lxml')
infobox = p_soup.find(class_=["infobox vcard", "infobox biography vcard"])
print(infobox)

<table class="infobox biography vcard" style="width:22em">
<tr>
<th colspan="2" style="text-align:center;font-size:125%;font-weight:bold"><span class="fn">Mary Anne MacLeod Trump</span></th>
</tr>
<tr>
<td colspan="2" style="text-align:center"><a class="image" href="/wiki/File:Mary_Anne_Trump.jpg"><img alt="Mary Anne Trump.jpg" data-file-height="300" data-file-width="205" height="307" src="//upload.wikimedia.org/wikipedia/en/0/0e/Mary_Anne_Trump.jpg" width="210"/></a>
<div>Mary MacLeod in 1935</div>
</td>
</tr>
<tr>
<th scope="row">Born</th>
<td><span class="nickname">Mary Anne MacLeod</span><br/>
<span style="display:none">(<span class="bday">1912-05-10</span>)</span>May 10, 1912<br/>
<span class="birthplace"><a href="/wiki/Tong,_Lewis" title="Tong, Lewis">Tong</a>, <a class="mw-redirect" href="/wiki/Isle_of_Lewis" title="Isle of Lewis">Isle of Lewis</a>, <a href="/wiki/Scotland" title="Scotland">Scotland</a>, <a href="/wiki/United_Kingdom_of_Great_Britain_and_Ireland" title="United K